In [1]:
import pandas as pd
import re
import numpy as np
# from pyspark import SparkContext, SparkConf
# from pyspark.sql import SparkSession

In [12]:
path = 'sample/GL 5 6 7 8 classification.xlsx'
df = pd.read_excel(path, sheet_name=0, header=None)
# new_header = df.iloc[0] #grab the first row for the header
# df = df[1:]
df.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,No.,Cost Center,NaN,G/L Account,NaN,Journal Entry,Posting Date,Expense/Income,Header Text,Journal Entry Item,...,Reference Source Document Type,Company Currency,NaN,Line Item Currency,NaN,Transaction Currency,NaN,Transaction Currency Amount,Line Item Currency Amount,Company Currency Amount
2,1,A00010,Common,642700,[Adm] Telephone charge,200000000586,29.04.2022,Expense,Telephone charge,2,...,Supplier Invoice,VND,Dong,VND,Dong,VND,Dong,22000,22000,22000
3,2,A00010,Common,642700,[Adm] Telephone charge,200000000639,29.04.2022,Expense,Telephone charge,2,...,Supplier Invoice,VND,Dong,VND,Dong,VND,Dong,1158053,1158053,1158053
4,3,A00010,Common,642700,[Adm] Telephone charge,200000000640,29.04.2022,Expense,Telephone charge,2,...,Supplier Invoice,VND,Dong,VND,Dong,VND,Dong,409625,409625,409625


In [14]:
new_header = df.iloc[1] #grab the first row for the header
df = df[2:]
df.columns = new_header
df.head()

1,No.,Cost Center,NaN,G/L Account,NaN,Journal Entry,Posting Date,Expense/Income,Header Text,Journal Entry Item,...,Reference Source Document Type,Company Currency,NaN,Line Item Currency,NaN,Transaction Currency,NaN,Transaction Currency Amount,Line Item Currency Amount,Company Currency Amount
2,1,A00010,Common,642700,[Adm] Telephone charge,200000000586,29.04.2022,Expense,Telephone charge,2,...,Supplier Invoice,VND,Dong,VND,Dong,VND,Dong,22000,22000,22000
3,2,A00010,Common,642700,[Adm] Telephone charge,200000000639,29.04.2022,Expense,Telephone charge,2,...,Supplier Invoice,VND,Dong,VND,Dong,VND,Dong,1158053,1158053,1158053
4,3,A00010,Common,642700,[Adm] Telephone charge,200000000640,29.04.2022,Expense,Telephone charge,2,...,Supplier Invoice,VND,Dong,VND,Dong,VND,Dong,409625,409625,409625
5,4,A00010,Common,642700,[Adm] Telephone charge,200000000641,29.04.2022,Expense,Telephone charge,2,...,Supplier Invoice,VND,Dong,VND,Dong,VND,Dong,44000,44000,44000
6,5,A00010,Common,642700,[Adm] Telephone charge,200000001576,05.05.2022,Expense,Telephone charge,2,...,Supplier Invoice,VND,Dong,VND,Dong,VND,Dong,1263764,1263764,1263764


In [50]:
kw = pd.read_excel(path, sheet_name=1)
kw = kw[kw['Contain'].notnull()].fillna('x')
keyword = list(kw['Contain'].str.lower())
keyword_1= list(kw['Return 1'].str.lower())
keyword_2= list(kw['Return 2'].str.lower())
keyword_2
# change_kw = list(kw['Text return'])
# keyword

['to clarify',
 'accruals',
 'x',
 'to clarify',
 'to clarify',
 'deductible',
 'deductible',
 'deductible',
 'to clarify',
 'x',
 'to clarify',
 'deductible',
 'to clarify',
 'non-deductible',
 'to clarify',
 'to clarify',
 'deductible',
 'non-deductible',
 'to clarify',
 'to clarify',
 'deductible',
 'deductible',
 'to clarify',
 'deductible',
 'non-deductible',
 'non-deductible',
 'x',
 'deductible',
 'deductible',
 'deductible',
 'deductible',
 'x',
 'to clarify',
 'deductible',
 'to clarify',
 'to clarify',
 'deductible',
 'to clarify',
 'to clarify',
 'to clarify',
 'to clarify',
 'to clarify',
 'to clarify',
 'to clarify',
 'deductible',
 'to clarify',
 'deductible',
 'to clarify',
 'to clarify',
 'deductible',
 'to clarify',
 'to clarify',
 'to clarify',
 'to clarify',
 'deductible',
 'to clarify',
 'to clarify',
 'to clarify',
 'to clarify',
 'to clarify',
 'deductible',
 'to clarify',
 'to clarify',
 'to clarify',
 'to clarify',
 'deductible',
 'to clarify',
 'to clarify',
 '

In [73]:
df1 = df[['Header Text']].astype(str)
df1['Header Text'] = df['Header Text'].str.lower()
df1.fillna('NO_DATA', inplace=True)

In [74]:
df1.head()

1,Header Text
2,telephone charge
3,telephone charge
4,telephone charge
5,telephone charge
6,telephone charge


In [75]:
def return_keyword(column, key_words):
    for i in key_words:
        if i in column:
            return i
            # return i[0].upper()+ i[1:]
    return ''
    
df1['return_kw'] = df1['Header Text'].apply(return_keyword, key_words= keyword)
df1['return_kw1'] = df1['Header Text'].apply(return_keyword, key_words= keyword)
# df1[df1['return_kw'] != '']
df1.tail(20)

1,Header Text,return_kw,return_kw1
72226,flu vaccination for employees,,
72227,pay clear jp trip 04/11-11/11/2022 trang,,
72228,pay clear jp trip 04/11-11/11/2022 ha,,
72229,"pay fct 31/08/2020, late pay-cvf444",fct,fct
72230,"pay fct 22/10/2020, late pay-cvf559",fct,fct
72231,expense for first aid training,,
72232,pay hn trip 22/09/2022,,
72233,pay tools for pccc festival-vu fac,,
72234,pay make lockers keys - huong ad,,
72235,"pay visa fee, fax, notarized, photo fee",,


In [76]:
new_dict = {k: v for k, v in zip(keyword, keyword_2)}
new_dict1 = {k: v for k, v in zip(keyword, keyword_2)}
df2 = df1.replace({'return_keyword': new_dict})
df2 = df2.replace({'return_kw1': new_dict1})
df2.tail()

1,Header Text,return_kw,return_kw1
72241,pay fct & late payment on 14/1 & 21/2/20,fct,deductible
72242,pay fct & late payment on 14/1 & 21/2/20,fct,deductible
72243,pay fct & late payment on 29/9/2022,fct,deductible
72244,estimated cit in oct.22,estimated cit,deductible
72245,estimated cit in nov.2022,estimated cit,deductible


In [77]:
df_return = pd.concat([df, df2], axis=1)
df_return.shape
df_return[df_return['return_kw'] != '']

1,No.,Cost Center,NaN,G/L Account,NaN,Journal Entry,Posting Date,Expense/Income,Header Text,Journal Entry Item,...,Line Item Currency,NaN,Transaction Currency,NaN,Transaction Currency Amount,Line Item Currency Amount,Company Currency Amount,Header Text,return_kw,return_kw1
2,1,A00010,Common,642700,[Adm] Telephone charge,200000000586,29.04.2022,Expense,Telephone charge,2,...,VND,Dong,VND,Dong,22000,22000,22000,telephone charge,telephone,deductible
3,2,A00010,Common,642700,[Adm] Telephone charge,200000000639,29.04.2022,Expense,Telephone charge,2,...,VND,Dong,VND,Dong,1158053,1158053,1158053,telephone charge,telephone,deductible
4,3,A00010,Common,642700,[Adm] Telephone charge,200000000640,29.04.2022,Expense,Telephone charge,2,...,VND,Dong,VND,Dong,409625,409625,409625,telephone charge,telephone,deductible
5,4,A00010,Common,642700,[Adm] Telephone charge,200000000641,29.04.2022,Expense,Telephone charge,2,...,VND,Dong,VND,Dong,44000,44000,44000,telephone charge,telephone,deductible
6,5,A00010,Common,642700,[Adm] Telephone charge,200000001576,05.05.2022,Expense,Telephone charge,2,...,VND,Dong,VND,Dong,1263764,1263764,1263764,telephone charge,telephone,deductible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72241,72240,#,Not assigned,811000,Miscellaneous expenses,250000001795,30.11.2022,Expense,Pay FCT & late payment on 14/1 & 21/2/20,3,...,VND,Dong,VND,Dong,16722454,16722454,16722454,pay fct & late payment on 14/1 & 21/2/20,fct,deductible
72242,72241,#,Not assigned,811000,Miscellaneous expenses,250000001805,30.11.2022,Expense,Pay FCT & late payment on 14/1 & 21/2/20,3,...,VND,Dong,VND,Dong,16168091,16168091,16168091,pay fct & late payment on 14/1 & 21/2/20,fct,deductible
72243,72242,#,Not assigned,811000,Miscellaneous expenses,250000001806,30.11.2022,Expense,Pay FCT & late payment on 29/9/2022,3,...,VND,Dong,VND,Dong,20943,20943,20943,pay fct & late payment on 29/9/2022,fct,deductible
72244,72243,#,Not assigned,821100,Corporate income tax,1295,31.10.2022,Expense,Estimated CIT in Oct.22,1,...,VND,Dong,VND,Dong,2382426912,2382426912,2382426912,estimated cit in oct.22,estimated cit,deductible


In [97]:
df_return.to_csv('GL_return.csv', index=False, encoding='utf8')

In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName("Python Spark DataFrames basic example").config("spark.some.config.option", "some-value").getOrCreate()

df = spark.read.option('header', True).csv('GL_return.csv', )
df.show()

AnalysisException: Cannot resolve column name "Text" among (Company Code, Posting period, Ledger, Reference, Document type, Document Header Text, Document Number, Text7, Document Currency Key, Document Currency Value, Company Code Currency Key, Company Code Currency Value, G/L Account, G/L Account: Long Text, Country General Ledger Account, Country G/L Account Description, Reporting Acct, Reporting Account Description, Business Area, Profit Center, Profit Center: Long Text, Channel, Channel Description, Cost Center, Cost Center: Long Text, RI Code, RICODE Description, Risk Location, Risk Location Description, Code Block 1, Code Block 1 Description, Trading partner, Und YR, CYPY, Acc YR, Cat Code, Catastrophe Code Description, Transaction type, Group Currency Key, Group Currency Value, Global Company Currency Key, Global Company Currency Value, Reversal Reason, Reversed With, Reverse Posting Date, Entry Date, Parked By, User Name, Tax Code ( MWSK1 ), Tax code ( MWSKZ ), EY Classificatio

(99478, 2)